In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lmfit import Model
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
from scipy.stats.distributions import t

# Multiple independent variables

***Non-linear Regression Analysis to Determine the Model Parameters k, Kb , and Kt***

You can download Fogler's book from here:
https://doku.pub/download/h-scott-fogler-essentials-of-chemical-reaction-engineering-z0x2ze484wqn

# 1st method using lmfit

In [2]:
# observed experimental reaction rates for the reaction C6H5CH3 + H2 → C6H6 + CH4 or t + h → b + m
data = pd.DataFrame({"r":[71,71.3,41.6,19.7,42.0,17.1,71.8, 
142.0,284.0,47.0,71.3,117.0,127.0,131.0,133.0,41.8], 
"t":[1,1,1,1,1,1,1,1,1,0.5,1,5,10,15,20,1], "h":[1,1,1,1,1,1,1,2,4,1,1,1,1,1,1,1],
"m":[1,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1], "b":[0,0,1,4,1,5,0,0,0,0,0,0,0,0,0,1]
})

# multiple independent variables - partial pressures of components
x_data = np.array([data["t"], data["h"], data["b"], data["m"]])

# dependent variable - rate of reaction
y_observed = data["r"]

In [3]:
# calculate y
def calc_y(x, k, Kb, Kt):
    
    num = k*x[0]*x[1]
    denom = 1+Kb*x[2]+Kt*x[0]
    y = num/denom #(mol/kg_cat.s)
    
    return y

my_model = Model(calc_y)

# set the boundries for each parameter
my_model.set_param_hint('k', min=0, max=200)
my_model.set_param_hint('Kb', min=0, max=150)
my_model.set_param_hint('Kt', min=0, max=150)

# initial values
result = my_model.fit(y_observed, x=x_data, k=0.05, Kb=200.0, Kt=1.0)

print(result.fit_report())

best_values = result.best_values
best_fit = result.best_fit

[[Model]]
    Model(calc_y)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 94
    # data points      = 16
    # variables        = 3
    chi-square         = 3.26050902
    reduced chi-square = 0.25080839
    Akaike info crit   = -19.4512864
    Bayesian info crit = -17.1335202
[[Variables]]
    k:   144.767308 +/- 0.57427659 (0.40%) (init = 0.05)
    Kb:  1.39052641 +/- 0.02120285 (1.52%) (init = 150)
    Kt:  1.03841056 +/- 0.00609291 (0.59%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(k, Kt)  = 0.957
    C(k, Kb)  = 0.343
    C(Kb, Kt) = 0.309


# Second method using Scipy

In [4]:
y_observed = data["r"]

def Rate(x, k, Kb, Kt):
    
    num = k*x[0]*x[1]
    denom = 1+Kb*x[2]+Kt*x[0]
    y = num/denom #(mol/kg_cat.s)
    
    return y

    
popt, pcov = curve_fit(Rate, x_data, y_observed, p0 =[1.0, 1.0, 1.0], 
bounds = [0, [np.inf, np.inf, np.inf]])

#k value needs to be multiplied by 10^-10
k, Kb, Kt = popt

k_rate_constant = k*1e-10

residuals = Rate(x_data, *popt) - y_observed
rss = sum(residuals**2)

y_mean = np.mean(y_observed)
total_residuals = y_observed - y_mean
tss = sum(total_residuals**2)

n = len(y_observed) # number of data points
p = len(popt) # number of parameters
alpha = 0.05

dof = max(0, n - p) #number of degrees of freedom

r_squared = 1 - (rss/tss)
r_adjusted = 1 - ((1 - r_squared)*(n - 1))/(n - p - 1)

tval = t.ppf(1.0-alpha/2., dof)

#confidence level for the estimated parameters
for i, j, var in zip(range(n), popt, np.diag(pcov)):
    sigma = var**0.5
    print('j{0}: {1} [{2}  {3}]'.format(i, j, j - sigma*tval, j + sigma*tval))
    
variance = rss/dof

rmse = np.sqrt(variance)

print("The rate constant value is: ", k_rate_constant)
print("The equilibrium constant of product b is", Kb)
print("The equilibrium constant of product t is", Kt)
print("The R squared value is:", r_squared)
print("The R_adjusted value is: ", r_adjusted)
print("The root mean square error (RMSE) or (RMSD) is", rmse)
print("The variance is", variance)

j0: 144.76730767069233 [143.5266570647402  146.00795827664447]
j1: 1.3905263980717728 [1.3447203893734294  1.4363324067701162]
j2: 1.0384105567359774 [1.0252475851572633  1.0515735283146914]
The rate constant value is:  1.4476730767069233e-08
The equilibrium constant of product b is 1.3905263980717728
The equilibrium constant of product t is 1.0384105567359774
The R squared value is: 0.9999509157577747
The R_adjusted value is:  0.9999386446972184
The root mean square error (RMSE) or (RMSD) is 0.500807733623429
The variance is 0.25080838605703537
